In [1]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Configuration constants
WEAVIATE_URL = "https://eypxka08rk6gbnyt57idzq.c0.us-west3.gcp.weaviate.cloud"
WEAVIATE_API_KEY = "jWgkVYQprMDiOuvTh56iLqRK5oqITcxN27wJ"
DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [6]:
# Setup vector store
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [3]:
SYSTEM_PROMPT = PromptTemplate(
    template=(
                "Dựa trên tài liệu đã trích xuất, hãy phân tích và trả lời câu hỏi.\n"
                "Câu hỏi: {question}\n"
                "Tài liệu: {context}\n"
                "Hãy suy nghĩ từng bước:\n"
                "1. Phân tích xem thông tin có đủ và liên quan không?\n"
                "2. Nếu chưa đủ, hãy đưa ra truy vấn mới để tìm thêm thông tin\n"
                "3. Nếu đã đủ, đưa ra câu trả lời cuối cùng\n\n"
                "Hãy trả lời theo định dạng sau:\n"
                "Phân tích: <phân tích thông tin hiện có>\n"
                "Quyết định: [Cần thêm thông tin/Đã đủ thông tin]\n"
                "Truy vấn tiếp theo: <truy vấn mới> (nếu cần)\n"
                "Câu trả lời cuối cùng: <câu trả lời> (nếu đã đủ thông tin)\n"
            )
)

In [7]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True),
)

In [10]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=5,
    alpha=0.3  # Balance between keyword and semantic search
)

In [7]:
from llama_index.llms.openai import OpenAI

In [8]:
llm = OpenAI(model="gpt-4o-mini", api_key="")

In [9]:
# Test reasoning and retrieval
async def test_autonomous_reasoning(question):
    retrieved_docs = retriever.retrieve(question)
    docs_text = "\n".join([doc.text for doc in retrieved_docs])
    print(docs_text)
    prompt = SYSTEM_PROMPT.format(question=question, context=docs_text)
    response = await llm.acomplete(prompt)
    return response.text.strip()


In [14]:
import asyncio

question = "Xe vượt tải trọng cho phép từ 10% đến 20% bị phạt bao nhiêu tiền?"

# Use `await` directly in Jupyter Notebook
answer = await test_autonomous_reasoning(question)

print("AI Answer:", answer)


NameError: name 'test_autonomous_reasoning' is not defined

In [12]:
question = "Tôi bị phạt bao nhiêu khi chạy xe máy 70km/h, vượt đèn đỏ và có nồng độ cồn 0.3mg/L?"
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : mô_tô , gắn máy mức phạt : tước quyền sử_dụng giấy_phép lái_xe từ 22 tháng đến 24 tháng nội_dung vi_phạm : không chấp_hành yêu_cầu kiểm_tra về chất ma_túy hoặc chất kích_thích khác mà pháp_luật cấm sử_dụng

Metadata: {'category': 'mô tô, gắn máy', 'fine_amount': 'Tước quyền sử dụng giấy phép lái xe từ 22 tháng đến 24 tháng', 'violation_type': 'nồng_độ_cồn', 'original_text': 'Loại phương tiện: mô tô, gắn máy\nMức phạt: Tước quyền sử dụng giấy phép lái xe từ 22 tháng đến 24 tháng\nNội dung vi phạm: Không chấp hành yêu cầu kiểm tra về chất ma túy hoặc chất kích thích khác mà pháp luật cấm sử dụng'}

Score: 0.920668363571167

Result 2:

Text: loại phương_tiện : mô_tô , gắn máy mức phạt : phạt tiền từ 8.000.000 đồng đến 10.000.000 đồng_nội_dung vi_phạm : không chấp_hành yêu_cầu kiểm_tra về chất ma_túy hoặc chất kích_thích khác mà pháp_luật cấm sử_dụng

Metadata: {'category': 'mô tô, gắn máy', 'fine_amount': 'Phạt tiền từ 8.000.000 đồng đến 10.000.000 đồng